In [104]:
import csv
import pandas as pd
import numpy as np


In [19]:
df = pd.read_csv('C:/git_project/natural/data/okt_x_train.csv',encoding='utf-8-sig')
df
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df

,0
0,"['더빙', '진짜', '짜증나다', '목소리']"
1,"['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기'..."
2,"['너무', '재밓었다', '보다', '추천', '하다']"
3,"['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', ..."
4,"['사이', '몬', '페그', '익살스럽다', '연기', '돋보이다', '영화',..."
...,...
145388,"['인간', '문제', '지', '소', '는', '뭔', '죄인']"
145389,"['평점', '너무', '낮다']"
145390,"['게', '뭐', '요', '한국인', '은', '거들다', '먹거리', '고',..."
145391,"['청춘', '영화', '최고봉', '방황', '우울하다', '날', '자화상']"


# Data Preprocessing

In [90]:
train_data = pd.read_table('C:/git_project/natural/data/ratings_train.txt')   # 데이터 불러오기
test_data = pd.read_table('C:/git_project/natural/data/ratings_test.txt')

In [111]:
# train_data.loc[train_data['document'].duplicated()]     # 중복확인

In [66]:
# train_data.drop_duplicates(subset=['document'], inplace=True)      #중복제거

In [67]:
# train_data['document'].duplicated().values.any()            #중복확인

False

In [100]:
train_data.dropna(how='any', inplace=True)
test_data.dropna(how='any', inplace=True)

In [101]:
train_data.document.isnull().values.any(), test_data.document.isnull().values.any()

(False, False)

In [102]:
train_data[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [103]:
test_data[:3]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [72]:
train_data["document"][0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [73]:
# def tokenize(text):
#     return list(text)

# train_text = [tokenize(text) for text in train_data["document"]]
# test_text = [tokenize(text) for text in test_data["document"]]

In [74]:
# reverse_word_index = ["<PAD>", "<UNK>"] + list(set([word for text in train_text for word in text]))
# reverse_word_index = {wid: word for wid, word in enumerate(reverse_word_index)}

In [75]:
# word_index = {word: wid for wid, word in reverse_word_index.items()}

In [76]:
def encode_text(text, max_seq_len):
    coded = []
    for i in range(max_seq_len):
        if i < len(text):
            coded.append(word_index.get(text[i], word_index["<UNK>"]))
        else:
            coded.append(word_index["<PAD>"])
    return coded

def decode_text(coded):
    text = []
    for wid in coded:
        if wid != word_index["<PAD>"]:
            text.append(reverse_word_index.get(wid, "?"))
    return text

train_x = np.array([encode_text(text, 32) for text in train_text])
test_x = np.array([encode_text(text, 32) for text in test_text])

train_x.shape, test_x.shape

((146182, 32), (49997, 32))

In [77]:
" ".join(decode_text(train_x[0]))

'아   더 빙 . .   진 짜   짜 증 나 네 요   목 소 리'

In [105]:
train_y = np.array(train_data["label"])
test_y = np.array(test_data["label"])
train_y, test_y

(array([0, 1, 0, ..., 0, 1, 0], dtype=int64),
 array([1, 0, 0, ..., 0, 0, 0], dtype=int64))

# Model Building

In [106]:
from tensorflow import keras

vocab_size = len(word_index)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32, mask_zero=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(16, activation='tanh'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          96192     
                                                                 
 dropout_3 (Dropout)         (None, None, 32)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              37632     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 16)                2064      
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                      

In [107]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training

In [108]:
val_x = test_x[:]
val_y = test_y[:]

In [109]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(256)
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

ValueError: Dimensions 146182 and 149995 are not compatible

In [110]:
history = model.fit(train_dataset, epochs=1, validation_data=val_dataset, verbose=1)

572/572 [==============================] - 43s 66ms/step - loss: 0.5157 - accuracy: 0.7429 - val_loss: 0.4836 - val_accuracy: 0.7682


# Evaluation

In [112]:
model.predict([encode_text(tokenize("이거 잼있네"), 32)])[0]

array([0.6681564], dtype=float32)

In [113]:
model.predict([encode_text(tokenize("이거 졸잼이네"), 32)])[0]

array([0.09588921], dtype=float32)

In [114]:
model.predict([encode_text(tokenize("이거 개꿀잼"), 32)])[0]

array([0.65291137], dtype=float32)

In [115]:
model.predict([encode_text(tokenize("이거 노잼이네"), 32)])[0]

array([0.22635785], dtype=float32)

In [116]:
model.predict([encode_text(tokenize("완전 핵노잼"), 32)])[0]

array([0.39120466], dtype=float32)

In [117]:
results = model.evaluate(test_x, test_y, verbose=2)

1563/1563 - 8s - loss: 0.4836 - accuracy: 0.7682 - 8s/epoch - 5ms/step


In [118]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

# EDA

In [ ]:
# import matplotlib.pyplot as plt

# acc = history_dict['accuracy']
# val_acc = history_dict['val_accuracy']
# loss = history_dict['loss']
# val_loss = history_dict['val_loss']

# epochs = range(1, len(acc) + 1)

# # "bo"는 "파란색 점"입니다
# plt.plot(epochs, loss, 'bo', label='Training loss')
# # b는 "파란 실선"입니다
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.show()

In [ ]:
# plt.clf()   # 그림을 초기화합니다

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()